# Zadanie 1.

In [1]:
@generated function harmonic_mean_gen(x...)
    return harmonic_mean_gen_impl(x...)
end

harmonic_mean_gen (generic function with 1 method)

In [2]:
function harmonic_mean_gen_impl(x...)
    N = length(x)
    elNum = :($N)
    sum = :(0)
    for i = 1:N
        sum = :((1 / x[$i]) + $sum)
    end
    return :($elNum / $sum)
end

harmonic_mean_gen_impl (generic function with 1 method)

In [3]:
harmonic_mean_gen_impl(Int, Int, Int)

:(3 / (1 / x[3] + (1 / x[2] + (1 / x[1] + 0))))

In [4]:
harmonic_mean_gen_impl(2, 2, 5, 7)

:(4 / (1 / x[4] + (1 / x[3] + (1 / x[2] + (1 / x[1] + 0)))))

In [5]:
harmonic_mean_gen(2, 2, 5, 7)

LoadError: [91mMethodError: no method matching harmonic_mean_gen_impl(::Type{Int64}, ::Type{Int64}, ::Type{Int64}, ::Type{Int64})
The applicable method may be too new: running in world age 21836, while current world is 21837.[0m
Closest candidates are:
  harmonic_mean_gen_impl(::Any...) at In[2]:2 (method too new to be called from this world context.)[39m

In [6]:
harmonic_mean_gen(2, 2, 5)

LoadError: [91mMethodError: no method matching harmonic_mean_gen_impl(::Type{Int64}, ::Type{Int64}, ::Type{Int64})
The applicable method may be too new: running in world age 21836, while current world is 21837.[0m
Closest candidates are:
  harmonic_mean_gen_impl(::Any...) at In[2]:2 (method too new to be called from this world context.)[39m

# Zadanie 2.

In [7]:
dump(:(x + x))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol x
    3: Symbol x
  typ: Any


In [8]:
function autodiff(ex::Expr)::Expr
    if (ex.head == :call) autodiff(Val{ex.args[1]}, ex) end
end

autodiff (generic function with 1 method)

In [9]:
function autodiff(n::Number) return 0 end
function autodiff(s::Symbol) return 1 end

autodiff (generic function with 3 methods)

In [ ]:
function autodiff(::Type{Val{:+}}, ex::Expr)::Expr
    reduce((x,y) -> :($x + $y), map(autodiff, ex.args[2:end]))
end

function autodiff(::Type{Val{:-}}, ex::Expr)::Expr
    reduce((x,y) -> :($x - $y), map(autodiff, ex.args[2:end]))
end

function autodiff(::Type{Val{:*}}, ex::Expr)
    res = :(0)
    args = ex.args[2:end]
    argsdiff = map(autodiff, args)
    N = length(args)
    
    for i = 1:N
        part = 1
        for j = 1:N
            if i == j 
                ad = argsdiff[j]
                part = :($part * $ad)
            else 
                a = args[j]
                part = :($part * $a)
            end
        end
        res = :($res + $part)
    end
    res
end

function autodiff(::Type{Val{:/}}, ex::Expr)
    f = ex.args[2]
    g = ex.args[3]
    df = autodiff(ex.args[2])
    dg = autodiff(ex.args[3])
    :(($df * $g - $f * $dg) / ($g * $g))
end

In [ ]:
autodiff(:(x+2))

In [ ]:
autodiff(:(x-2))

In [ ]:
autodiff(:(5 * x * x))

In [ ]:
autodiff(:(5 / x + x * y))

In [ ]:
autodiff(:(x + x*y*z))

## Zadanie bonusowe.

In [ ]:
using DualNumbers
function autodiffFun(f) 
    g(x) = dualpart(f(Dual(x,1)))
end

In [ ]:
f(x) = x * x + x - 10x
g = autodiffFun(f)
g(5) == 1